In [26]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score as ACC
from sklearn.metrics import matthews_corrcoef as MCC
from sklearn.metrics import roc_auc_score as AUC
from sklearn.metrics import f1_score as F1
def auc_roc(y_test1,pre):
    pre=[p for p in pre[:,1]]
    #print(pre)
    fpr,tpr,threshold = roc_curve(y_test1,pre)###计算真正率和假正率
    roc_auc = auc(fpr,tpr)
    #print('roc_auc:%2f'%(roc_auc))
    return roc_auc
def analyze(pre,real):
    AUC_test=auc_roc(real,pre)
    pre=np.argmax(pre,axis=1)
    #print(pre)
    m=confusion_matrix(pre,real, labels=[0,1], sample_weight=None)
    print(m)
    tp, fp, fn, tn = m.ravel()
    TPR=tp/(tp+fn)
    TNR=tn/(tn+fp)
    ACC_test=ACC(real,pre)
    #AUC_test=auc_roc(real,pre)
    MCC_test=MCC(real,pre)
    F1_test=F1(real,pre)
    print('敏感度TPR：%2f，特异度TNR：%2f，ACC：%2f，AUC：%2f,MCC：%2f,F1：%2f'%(TPR,TNR,ACC_test,AUC_test,MCC_test,F1_test))

In [23]:
t_train=pd.read_csv('t_2_train.csv')
train_labels=list(t_train.iloc[:,-1])
t_train

,SMILES,terminal t1/2 (h),label
0,[Gd+3].COCCNC(=O)CN(CCN(CCN(CC([O-])=O)CC(=O)N...,1.46,0
1,COc1ccc2CN(C)CC[C@@]34C=C[C@H](O)C[C@@H]3Oc1c24,5.30,1
2,C[C@H](\C=C\[C@H](C)C(C)(C)O)[C@H]1CC[C@H]2\C(...,5.30,1
3,c1(n(c2c(C(=O)c3ccccc3C2=O)[n+]1Cc1cnccn1)CCOC)C,24.80,1
4,COC(=O)[C@H](c1ccccc1Cl)N2CCc3c(ccs3)C2,5.06,1
...,...,...,...
1060,CC(C)[C@@H](CC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3[C...,9.50,1
1061,CC1=C(C=C(C#N)C(=O)N1)c2ccncc2,0.80,0
1062,c1ccc(cc1)CC[C@H](N[C@@H]1C(N2[C@@H](CCC[N@@]2...,50.60,1
1063,c1cc(ccc1O)[C@@H](CN(C)C)C1(CCCCC1)O,8.40,1


In [36]:
train_finger_0=pd.read_csv('wm/train_finger_name.csv')
x_train_0=train_finger_0.iloc[:,1:]
x_train_0

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,24.916749,21.330173,3,3,NaN,NaN,NaN,NaN,NaN,NaN,...,9.680906,70.382546,662.154671,9.459352,3500004456,40,150.0,156.0,NaN,8.333333
1,16.944803,13.660186,0,1,27.577593,2.628402,5.093440,27.577593,1.313219,4.021546,...,10.368761,70.880960,287.152144,6.836956,766,44,122.0,153.0,6.340278,4.500000
2,23.434942,18.951645,0,0,36.526304,2.543268,4.979309,36.526304,1.217543,4.325897,...,10.440419,80.323551,416.329045,5.626068,2688,49,160.0,188.0,12.013889,6.347222
3,21.136972,16.996168,0,1,35.775182,2.598451,5.087521,35.775182,1.325007,4.241037,...,10.420822,77.262212,363.145167,7.894460,1672,49,146.0,179.0,8.361111,6.027778
4,16.280035,14.176323,0,1,27.584895,2.443323,4.860804,27.584895,1.313566,3.981092,...,9.976599,68.561247,321.059027,8.677271,867,33,110.0,131.0,6.527778,4.722222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,33.310425,24.004339,1,3,53.357565,2.636274,5.232660,53.357565,1.240874,4.681612,...,10.883748,95.225181,627.464493,5.809856,7704,75,228.0,271.0,16.298611,9.347222
1061,12.118275,10.624530,0,0,20.388826,2.390361,4.780722,20.388826,1.274302,3.691943,...,9.618801,48.335122,211.074562,8.442982,425,24,80.0,93.0,5.555556,3.666667
1062,21.517512,17.462176,2,1,35.745023,2.447673,4.855946,35.745023,1.276608,4.246447,...,10.129068,71.062891,389.195071,7.076274,2077,47,142.0,165.0,9.500000,6.250000
1063,14.527174,12.678904,0,1,23.527792,2.430456,4.860911,23.527792,1.238305,3.858594,...,9.841665,52.309217,263.188529,5.981557,672,28,96.0,110.0,7.006944,4.194444


In [44]:
x_train_1=x_train_0.dropna(how='any',axis=1)
x_train=np.array(x_train_1)

In [47]:
finger_name=list(x_train_1)

In [19]:
t_test=pd.read_csv('t_2_test.csv')
test_labels=list(t_test.iloc[:,-1])
t_test

,SMILES,terminal t1/2 (h),label
0,O([C@H]([C@H](NC(c1nc([C@@H](NC[C@@H](C(N)=O)N...,3.90,1
1,c1(nc(cc2ccccc12)C(=O)N([C@@H](CC)C)C)c1c(cccc...,3.70,1
2,[H]N([H])C([C@](CCN1CCCCC1)(CCN(C(C)C)C(C)=O)c...,12.00,1
3,Cc1cc(c(cc1C(=O)NC(=N)N)S(=O)(=O)C)n2cccc2,2.20,0
4,CCCN(CCC)S(=O)(=O)c1ccc(cc1)C(=O)O,5.90,1
...,...,...,...
261,N(CCN(CC)CC)C(C(=C(C1CNC=O)S2)C(=O)C(C2=CC=C2O...,6.19,1
262,n1(nnnc1SCC1=C(N2[C@H](OC1)[C@@](C2=O)(OC)NC(=...,0.60,0
263,OCCN1C[C@H](O)[C@@H](O)[C@H](O)[C@H]1CO,2.30,0
264,[C@@H]12[C@@H]([C@@]3(C(=CC(=O)CC3)CC2)C)CC[C@...,0.17,0


In [22]:
test_finger_0=pd.read_csv('wm/test_finger.csv')
test_finger=test_finger_0.iloc[:,1:]
test_finger

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,73.222407,55.918156,0,3,119.027589,2.484476,4.964968,119.027589,1.239871,5.463272,...,11.391808,153.940560,1414.518432,7.858436,62651,159,488.0,570.0,39.111111,21.305556
1,19.246361,15.961234,0,0,32.589361,2.475468,4.950937,32.589361,1.303574,4.144021,...,10.196717,59.921539,352.134241,7.655092,1410,43,130.0,155.0,8.750000,5.611111
2,21.027286,18.465823,0,1,34.361854,2.467430,4.934859,34.361854,1.227209,4.228365,...,10.193878,63.094999,407.233955,6.568290,1949,45,138.0,160.0,11.090278,6.333333
3,17.552118,15.944064,0,3,27.700033,2.474275,4.934782,27.700033,1.204349,4.047581,...,10.060021,70.484053,320.094311,8.423535,1153,35,118.0,137.0,9.840278,5.027778
4,13.807986,12.716102,1,0,22.785133,2.409629,4.819258,22.785133,1.199218,3.826585,...,9.757247,51.955014,285.103479,7.502723,728,29,90.0,103.0,8.506944,4.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,21.787059,17.061554,0,1,37.663781,2.525481,5.050961,37.663781,1.298751,4.279391,...,10.309386,64.462839,413.177313,7.378166,2274,49,146.0,173.0,9.750000,6.833333
262,24.332110,20.403834,3,0,39.998027,2.645787,5.291433,39.998027,1.249938,4.388472,...,10.733545,82.269503,496.064645,9.921293,3113,51,166.0,200.0,12.423611,7.250000
263,10.065904,9.843620,0,1,16.843988,2.396029,4.792057,16.843988,1.203142,3.528175,...,9.492432,45.488824,207.110673,6.680989,288,22,66.0,77.0,6.555556,3.361111
264,17.179861,13.863771,0,0,27.395456,2.609083,5.180348,27.395456,1.304546,4.029965,...,10.533402,70.727122,288.208930,5.881815,802,44,126.0,160.0,7.041667,4.305556


In [51]:
x_test=np.array(test_finger[finger_name])
x_test

array([[ 73.22240726,  55.91815633,   0.        , ..., 488.        ,
        570.        ,  21.30555556],
       [ 19.24636149,  15.96123414,   0.        , ..., 130.        ,
        155.        ,   5.61111111],
       [ 21.0272858 ,  18.46582344,   0.        , ..., 138.        ,
        160.        ,   6.33333333],
       ...,
       [ 10.06590388,   9.84361974,   0.        , ...,  66.        ,
         77.        ,   3.36111111],
       [ 17.17986114,  13.86377069,   0.        , ..., 126.        ,
        160.        ,   4.30555556],
       [ 16.16363102,  14.8248309 ,   2.        , ..., 110.        ,
        124.        ,   4.19444444]])

In [74]:
num_samples=int(x_train.shape[0]/3)
num_samples

355

In [167]:
# i=0
# clf0 = SVC(probability=True,C=3)#,C=3,gamma=0.0015)
# val_data= x_train[i*num_samples:(i+1)*num_samples,:]
# val_label= train_labels[i * num_samples:(i + 1) * num_samples]
# train_data=np.concatenate([x_train[:i*num_samples,:],x_train[(i+1)*num_samples:,:]],axis=0)
# train_label = np.concatenate([train_labels[:i*num_samples], train_labels[(i + 1) * num_samples:]], axis=0)
# clf0.fit(train_data,train_label)
# pre_clf=clf0.predict_proba(val_data)
# analyze(pre_clf,val_label)
# print('test')
# pre_clf0=clf0.predict_proba(x_test)
# analyze(pre_clf0,test_labels)

In [168]:
i=0
rf0=RandomForestClassifier( n_estimators=70,min_samples_leaf=6)#max_depth=3)#, , max_depth=12, min_samples_split=10,max_features=25 )#, min_samples_split=5,  
                                # min_samples_leaf=6,max_features=25 , random_state=10)
val_data= x_train[i*num_samples:(i+1)*num_samples,:]
val_label= train_labels[i * num_samples:(i + 1) * num_samples]
train_data=np.concatenate([x_train[:i*num_samples,:],x_train[(i+1)*num_samples:,:]],axis=0)
train_label = np.concatenate([train_labels[:i*num_samples], train_labels[(i + 1) * num_samples:]], axis=0)
rf0.fit(train_data,train_label)
pre_rf=rf0.predict_proba(val_data)
aa0=analyze(pre_rf,val_label)

probarf0=rf0.predict_proba(x_test)
aa=analyze(probarf0,test_labels)

[[ 69  51]
 [ 71 164]]
敏感度TPR：0.492857，特异度TNR：0.762791，ACC：0.656338，AUC：0.694219,MCC：0.264120,F1：0.728889
[[ 51  31]
 [ 57 127]]
敏感度TPR：0.472222，特异度TNR：0.803797，ACC：0.669173，AUC：0.733767,MCC：0.293538,F1：0.742690


In [154]:
i=2
xg0=XGBClassifier(n_estimators=200,max_depth=12,learning_rate=0.050)#,max_depth=5)#n_estimators=210)#,max_depth=7,learning_rate=0.105)
val_data= x_train[i*num_samples:(i+1)*num_samples,:]
val_label= train_labels[i * num_samples:(i + 1) * num_samples]
train_data=np.concatenate([x_train[:i*num_samples,:],x_train[(i+1)*num_samples:,:]],axis=0)
train_label = np.concatenate([train_labels[:i*num_samples], train_labels[(i + 1) * num_samples:]], axis=0)
xg0.fit(train_data,train_label)
pre_xg=xg0.predict_proba(val_data)
analyze(pre_xg,val_label)
print('test')
pre_xg0=xg0.predict_proba(x_test)
analyze(pre_xg0,test_labels)

[[ 79  43]
 [ 66 167]]
敏感度TPR：0.544828，特异度TNR：0.795238，ACC：0.692958，AUC：0.748309,MCC：0.351964,F1：0.753950
test
[[ 61  28]
 [ 47 130]]
敏感度TPR：0.564815，特异度TNR：0.822785，ACC：0.718045，AUC：0.766174,MCC：0.403406,F1：0.776119


In [109]:
i=1
gbm0=GradientBoostingClassifier(n_estimators=20)
val_data= x_train[i*num_samples:(i+1)*num_samples,:]
val_label= train_labels[i * num_samples:(i + 1) * num_samples]
train_data=np.concatenate([x_train[:i*num_samples,:],x_train[(i+1)*num_samples:,:]],axis=0)
train_label = np.concatenate([train_labels[:i*num_samples], train_labels[(i + 1) * num_samples:]], axis=0)
gbm0.fit(train_data,train_label)
pre_xg=gbm0.predict_proba(val_data)
analyze(pre_xg,val_label)
print('test')
pre_gbm0=gbm0.predict_proba(x_test)
analyze(pre_gbm0,test_labels)

[[ 71  41]
 [ 75 168]]
敏感度TPR：0.486301，特异度TNR：0.803828，ACC：0.673239，AUC：0.736023,MCC：0.307205,F1：0.743363
test
[[ 56  30]
 [ 52 128]]
敏感度TPR：0.518519，特异度TNR：0.810127，ACC：0.691729，AUC：0.743788,MCC：0.345050,F1：0.757396


In [141]:
pre=(probarf0+pre_gbm0+pre_xg0)
analyze(pre,test_labels)

[[ 57  30]
 [ 51 128]]
敏感度TPR：0.527778，特异度TNR：0.810127，ACC：0.695489，AUC：0.778833,MCC：0.353711,F1：0.759644
